# Exam Learning Path - Creation Tool

## Imports

In [ ]:
// Hard Coding Versions For Consistency
#r "nuget: FSharp.Data, 5.0.2"

Installed Packages FSharp.Data, 5.0.2

In [ ]:
open System.Linq
open FSharp.Data

## Domain

In [ ]:
type Exam = {
    Title: string
    DisplayName: string
    Link: string
    // State: ExamState
    LearningPaths: LearningPath list
}

and ExamState =
    | Current
    | Retired of string // Message

and LearningPath =
    { Title: string
      Link: string
      Duration: TimeSpan
      Modules: Module list }
      static member Default =
          { Title = "Missing"
            Link = "Missing"
            Duration = TimeSpan.FromSeconds(0)
            Modules = [] }

and Module =
    { Title: string
      Link: string
      Duration: TimeSpan }
      static member Default =
          { Title = "Missing"
            Link = "Missing"
            Duration = TimeSpan.FromSeconds(0) }

## Helpers

In [ ]:
let toAbsoluteURL (pageURL: string) (urlToUpdate: string) : string =
    let pageURI = Uri(pageURL)
    match Uri.TryCreate(pageURI, urlToUpdate) with
    | true, uri -> uri.ToString()
    | false, _  -> failwith $"urlToUpdate value is in an unexpected format. urlToUpdate = {urlToUpdate}"

### JSON

In [ ]:
module WebAPI =
    let [<Literal>] private ROOT_SEARCH_URL = "https://docs.microsoft.com/api/contentbrowser/search/certifications?locale=en-us&$orderBy=title&$top=30"
    let [<Literal>] private ROOT_EXAM_URL = "https://docs.microsoft.com/api/lists/studyguide/exam/exam.mb-340?locale=en-us"
    let [<Literal>] private ROOT_LEARNINGPATH_URL = "https://docs.microsoft.com/api/hierarchy/paths/learn.wwl.examine-core-capabilities-of-microsoft-dynamics-365-crm?locale=en-us"
    let [<Literal>] private ROOT_MODULE_URL = "https://docs.microsoft.com/api/hierarchy/paths/learn-dynamics.get-started-dynamics-365-commerce?locale=en-us"

    type SearchWebAPI = JsonProvider<ROOT_SEARCH_URL>
    type ExamWebAPI = JsonProvider<ROOT_EXAM_URL>
    type LearningPathWebAPI = JsonProvider<ROOT_LEARNINGPATH_URL>
    type ModuleWebAPI = JsonProvider<ROOT_MODULE_URL>


    let getSearchData () : SearchWebAPI.Result list =
        let staging = System.Collections.Generic.List<SearchWebAPI.Result>()
        
        let mutable url = ROOT_SEARCH_URL
        while not (String.IsNullOrWhiteSpace(url)) do
            url <- toAbsoluteURL ROOT_SEARCH_URL url

            let res = SearchWebAPI.AsyncLoad(url) |> Async.RunSynchronously
            staging.AddRange(res.Results)
            url <- res.NextLink

        staging.DistinctBy(fun x -> x.Uid)
        |> List.ofSeq

    let getExamData (examUid: string) : ExamWebAPI.Root =
        let url = ROOT_EXAM_URL.Replace("exam.mb-340", examUid)
        ExamWebAPI.AsyncLoad(url) |> Async.RunSynchronously

    let getLearningPathData (learningPathId: string) : LearningPathWebAPI.Root =
        let url = ROOT_LEARNINGPATH_URL.Replace("learn.wwl.examine-core-capabilities-of-microsoft-dynamics-365-crm", learningPathId)
        LearningPathWebAPI.AsyncLoad(url) |> Async.RunSynchronously

    let getModuleData (moduleId: string) : ModuleWebAPI.Root =
        let url = ROOT_LEARNINGPATH_URL.Replace("learn-dynamics.get-started-dynamics-365-commerce", moduleId)
        ModuleWebAPI.AsyncLoad(url) |> Async.RunSynchronously


## Crawl

Entry Point -> Domain Objects

In [ ]:
module Crawl =
    open System.Net

    let [<Literal>] ROOT_URL = "https://docs.microsoft.com/en-us/"

    let private webAPI_getLearningPathData id =
        try
            WebAPI.getLearningPathData id
            |> Some
        with
        | :? WebException -> None

    let crawl () =
        let searchData = WebAPI.getSearchData ()

        let searchCertificationData = searchData.Where(fun x -> x.ResourceType = "certification")
                                                .Select(fun x -> (x.Uid, x))
                                                |> Map.ofSeq

        let searchExamData = searchData.Where(fun x -> x.ResourceType = "examination")
                                       .Select(fun x -> (x.Uid, x))
                                       |> Map.ofSeq

        let examData = searchExamData.Keys
                                     .Select(fun uid -> (uid, WebAPI.getExamData uid))
                                     |> Map.ofSeq

        let examLearningPathData = examData.Values
                                           .SelectMany(fun x -> x.Items |> Seq.ofArray)
                                           .Select(fun x -> (x.Id, x))
                                           |> Map.ofSeq

        let learningPathData = examLearningPathData.Keys
                                                   .Select(fun id -> (id, webAPI_getLearningPathData id))
                                                   |> Map.ofSeq

        let learningPathModuleData = learningPathData.Values
                                                     .Where(Option.isSome)
                                                     .Select(Option.get)
                                                     .SelectMany(fun x -> x.Modules |> Seq.ofArray)
                                                     .Select(fun x -> (x.Uid, x))
                                                     |> Map.ofSeq

        let toAbsoluteURL = toAbsoluteURL ROOT_URL

        let modules = learningPathModuleData
                      |> Map.map (fun id x -> { Title = x.Title
                                                Link = toAbsoluteURL x.Url
                                                Duration = TimeSpan.FromMinutes(x.DurationInMinutes) })

        let learningPaths = learningPathData
                            |> Map.map (fun uid x -> match x with
                                                     | None -> LearningPath.Default
                                                     | Some x -> { Title = x.Title
                                                                   Link = toAbsoluteURL x.Url
                                                                   Duration = TimeSpan.FromMinutes(x.DurationInMinutes)
                                                                   Modules = x.Modules
                                                                              .Select(fun x -> modules[x.Uid])
                                                                              |> List.ofSeq })

        let exams = searchExamData
                    |> Map.map (fun uid x -> { Title = x.Title
                                               DisplayName = x.ExamDisplayName
                                               Link = toAbsoluteURL x.Url
                                               LearningPaths = (examData[uid]).Items
                                                                              .Select(fun x -> learningPaths[x.Id])
                                                                              |> List.ofSeq })

        exams.Values
             |> List.ofSeq

## Grouping

Group modules into weeks at some sustainable level

In [ ]:
module Grouping =

    let min = TimeSpan.FromMinutes(60)
    let max = TimeSpan.FromMinutes(90)

    let groupModulesIntoWeeks (exam: Exam) =
        let mutable dur = TimeSpan.FromSeconds(0)
        let mutable week = 1

        let flat = exam.LearningPaths
                       .SelectMany(fun lp -> lp.Modules
                                               .Select(fun m -> (lp, m)))

        seq {
            for (lp, m) in flat do
                if ((m.Duration > max || dur.Add(m.Duration) > max) && dur <> TimeSpan.FromSeconds(0)) then
                    dur <- TimeSpan.FromSeconds(0)
                    week <- week + 1

                dur <- dur.Add(m.Duration)

                yield (lp, m, week)
        }
        |> List.ofSeq

## File System

In [ ]:
module FileSystem =

    let initExamsDirectory () : unit =
        if Directory.Exists("Exams") then
            Directory.Delete("Exams", true)

        System.Threading.Thread.Sleep(250)

        Directory.CreateDirectory("Exams") |> ignore

    let examPath (exam: Exam) : string =
        $"Exams/{exam.DisplayName}.md"

    let readmePath = "README.md"

    let readmeTemplatePath = "README.md.template"

## Markdown

In [ ]:
module Markdown =

    let h1 str = "# " + str
    let boldStr str = "**" + str + "**"
    let boldInt i   = "**" + (string i) + "**"
    let url text url = "[" + text + "](" + url + ")"
    let pad str = " " + str + " "
    let emptyCell = " "

    let examFullName (exam: Exam) : string =
        $"{exam.DisplayName}: {exam.Title}"

    let exam (exam: Exam) : string =
        let sb = StringBuilder()
        
        sb.AppendLine(h1 $"{exam.DisplayName}: {exam.Title}")
          .AppendLine()
          .AppendLine($"**Exam/Learning Path:** {exam.Link}")
          .AppendLine()
          |> ignore

        sb.AppendLine("| **Learning Path** | **Module** | **Week** |")
          .AppendLine("|-|-|-|")
          |> ignore

        let mutable previousLeaarningPathLink = null
        for (lp, m, w) in Grouping.groupModulesIntoWeeks exam do
            let lpCell = if lp.Link = previousLeaarningPathLink then
                            emptyCell
                         else
                            previousLeaarningPathLink <- lp.Link
                            boldStr (url lp.Title lp.Link)

            let mCell = pad (url m.Title m.Link)

            let wCell = pad (boldInt w)

            sb.AppendLine($"|{lpCell}|{mCell}|{wCell}")
              |> ignore

        sb.ToString()

    let examsLinks (exams: Exam list) : string =
        let sb = StringBuilder()
        
        for exam in exams do
            sb.Append(url (examFullName exam) (FileSystem.examPath exam))
              .AppendLine("<br/>")
              |> ignore
        
        sb.ToString()

    let readme (readmeTemplate: string, exams: Exam list) : string =
        let examsLinksMarkdown = examsLinks exams
        readmeTemplate.Replace("{{{Exam-Links}}}", examsLinksMarkdown)

## Do

In [ ]:
let exams = Crawl.crawl()

In [ ]:
do
    FileSystem.initExamsDirectory ()

    for exam in exams do
        let path = FileSystem.examPath exam
        let markdown = Markdown.exam exam
        File.WriteAllText(path, markdown)

    let readmeTemplate = File.ReadAllText(FileSystem.readmeTemplatePath)
    let readme = Markdown.readme(readmeTemplate, exams)
    File.WriteAllText(FileSystem.readmePath, readme)